# RAG 평가하기

### 선수 조건
- Amazon Bedrock Knowledge base 를 먼저 생성 해야 합니다.
    - 참조 블로그: [Amazon Bedrock Knowledge base로 30분 만에 멀티모달 RAG 챗봇 구축하기 실전 가이드](https://aws.amazon.com/ko/blogs/tech/practical-guide-for-bedrock-kb-multimodal-chatbot/)
    - data: ../data/241231-Sonnet-recipe.pdf 문서를 사용하세요.

---    

## 0. 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)


module_path = ".."
add_python_path(module_path)


from libs.notebook_util import (
    pretty_print_df, convert_to_list, format_text,
    print_formatted_text, generate_answer,
    show_sample_row_eval,
    merge_dataset_and_results
)

python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock/notebook', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python310.zip', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/lib-dynload', '', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock']


/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. 합성 데이터 로딩
    


In [2]:
import pandas as pd

df = pd.read_json("../data/sample_qa_dataset.jsonl", lines=True)
print(df.shape)
df.head()

(10, 4)


,question,ground_truth,question_type,contexts
0,지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴...,지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴...,complex,"프랑스 요리로 더 살펴보면, 코코뱅은 닭고기를 레드와인에 장시간 끓여 만드는\n전통..."
1,팟타이는 어떤 재료로 만드나요?,팟타이는 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려 만듭니다.,simple,기와 달걀을 간장 베이스의 달달한 육수에 조려 밥 위에 올립니다.\n중식으로는 마파...
2,"한국, 이탈리아, 태국, 베트남 요리의 특징을 비교하며, 각 나라의 대표 요리에서 ...","한국, 이탈리아, 태국, 베트남 요리는 각각 고유한 특징을 가지고 있지만, 몇 가지...",complex,"볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다..."
3,라자냐는 어떤 재료로 만들어지나요?,"라자냐는 파스타면, 미트소스, 베샤멜 소스, 치즈를 층층이 쌓아 오븐에 구워 만듭니다.",simple,볶다가 뜨거운 육수를 조금씩 부어가며 천천히 익혀 만듭니다. 라자냐는 파스타\n면과...
4,"여러 나라의 요리법을 비교했을 때, 와인을 주요 재료로 사용하는 요리들의 공통점과 ...","와인을 주요 재료로 사용하는 요리들을 비교해보면, 프랑스의 비프 부르기뇽과 코코뱅이...",complex,"요리입니다. 어향육슬은 돼지고기를 채썰어 매콤새콤한 소스로 볶아내고, 팔보채\n는 ..."


In [3]:
# sample_df = df.iloc[[1]]
pretty_print_df(df.iloc[[0]])    
# df.head(1).iloc[[0]].contexts.iloc[0]

question:
지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성을 제안한다면, 어떤 요리들을 선택하고 그 이유는 무엇인가요?
ground_truth:
지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성은 다음과 같습니다: 1.
프랑스의 라타투이: 가지, 토마토, 호박, 파프리카 등 다양한 채소로 만든 채식 요리로, 채식주의자에게 적합합니다.
2.
스페인의 가스파초: 토마토, 오이, 피망으로 만든 차가운 스프로, 채식주의자도 즐길 수 있는 상큼한 요리입니다.
3.
그리스의 무사카: 가지와 다진 고기를 층층이 쌓아 만드는 요리로, 육식주의자를 위한 선택입니다.
채식 버전으로 변형도 가능합니다.
4.
모로코의 쿠스쿠스: 밀가루로 만든 알갱이에 채소를 얹어 먹는 요리로, 고기를 별도로 제공하여 선택적으로 먹을 수 있게 합니다.
5.
스페인의 빠에야: 해산물과 채소를 넣은 쌀 요리로, 해산물을 먹는 사람과 채식주의자 모두에게 적합합니다.
이러한 구성은 다양한 식단 선호도를 고려하면서도 지중해 요리의 특색을 잘 보여줍니다.
채식 옵션과 육류 옵션을 동시에 제공하여 모든 식성의 사람들을 만족시킬 수 있으며, 각 요리의 재료와 조리법을 조절하여 개인의 선호에 맞출 수 있는 유연성도 갖추고 있습니다.
question_type:
complex
contexts:
프랑스 요리로 더 살펴보면, 코코뱅은 닭고기를 레드와인에 장시간 끓여 만드는 전통 요리입니다.
와인의 풍미가 닭고기에 깊이 배어들어 독특한 맛을 자아내며, 양파와 버섯, 베이컨을 함께 넣어 풍성한 맛을 더합니다.
라타투이는 가지, 토마 토, 호박, 파프리카 등 지중해 채소들을 올리브유로 천천히 익혀 만드는 프로방 스 지방의 대표적인 채식 요리입니다.
끼슈는 버터가 듬뿍 들어간 파이 반죽 위 에 달걀과 크림을 섞은 커스터드를 붓고 베이컨, 치즈, 채소를 넣어 구워내는 고 소한 요리입니다.
스페인 요리에서는 빠에야가 유명한데, 사프란으로 노란빛을

## 2. Test Data Set 생성

#### Test Dataset 갯수 지정

In [4]:
subset_length=5
df.head(subset_length)

,question,ground_truth,question_type,contexts
0,지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴...,지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴...,complex,"프랑스 요리로 더 살펴보면, 코코뱅은 닭고기를 레드와인에 장시간 끓여 만드는\n전통..."
1,팟타이는 어떤 재료로 만드나요?,팟타이는 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려 만듭니다.,simple,기와 달걀을 간장 베이스의 달달한 육수에 조려 밥 위에 올립니다.\n중식으로는 마파...
2,"한국, 이탈리아, 태국, 베트남 요리의 특징을 비교하며, 각 나라의 대표 요리에서 ...","한국, 이탈리아, 태국, 베트남 요리는 각각 고유한 특징을 가지고 있지만, 몇 가지...",complex,"볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다..."
3,라자냐는 어떤 재료로 만들어지나요?,"라자냐는 파스타면, 미트소스, 베샤멜 소스, 치즈를 층층이 쌓아 오븐에 구워 만듭니다.",simple,볶다가 뜨거운 육수를 조금씩 부어가며 천천히 익혀 만듭니다. 라자냐는 파스타\n면과...
4,"여러 나라의 요리법을 비교했을 때, 와인을 주요 재료로 사용하는 요리들의 공통점과 ...","와인을 주요 재료로 사용하는 요리들을 비교해보면, 프랑스의 비프 부르기뇽과 코코뱅이...",complex,"요리입니다. 어향육슬은 돼지고기를 채썰어 매콤새콤한 소스로 볶아내고, 팔보채\n는 ..."


In [5]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df.head(subset_length))
test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

Map: 100%|██████████| 5/5 [00:00<00:00, 1248.60 examples/s]

Error: invalid syntax (<unknown>, line 1)
Error: invalid syntax (<unknown>, line 1)
Error: invalid syntax (<unknown>, line 1)
Error: invalid syntax (<unknown>, line 1)
Error: invalid syntax (<unknown>, line 1)
Dataset({
    features: ['question', 'ground_truth', 'question_type', 'contexts'],
    num_rows: 5
})


In [6]:
test_dataset[0]

{'question': '지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성을 제안한다면, 어떤 요리들을 선택하고 그 이유는 무엇인가요?',
 'ground_truth': '지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성은 다음과 같습니다:\n\n1. 프랑스의 라타투이: 가지, 토마토, 호박, 파프리카 등 다양한 채소로 만든 채식 요리로, 채식주의자에게 적합합니다.\n\n2. 스페인의 가스파초: 토마토, 오이, 피망으로 만든 차가운 스프로, 채식주의자도 즐길 수 있는 상큼한 요리입니다.\n\n3. 그리스의 무사카: 가지와 다진 고기를 층층이 쌓아 만드는 요리로, 육식주의자를 위한 선택입니다. 채식 버전으로 변형도 가능합니다.\n\n4. 모로코의 쿠스쿠스: 밀가루로 만든 알갱이에 채소를 얹어 먹는 요리로, 고기를 별도로 제공하여 선택적으로 먹을 수 있게 합니다.\n\n5. 스페인의 빠에야: 해산물과 채소를 넣은 쌀 요리로, 해산물을 먹는 사람과 채식주의자 모두에게 적합합니다.\n\n이러한 구성은 다양한 식단 선호도를 고려하면서도 지중해 요리의 특색을 잘 보여줍니다. 채식 옵션과 육류 옵션을 동시에 제공하여 모든 식성의 사람들을 만족시킬 수 있으며, 각 요리의 재료와 조리법을 조절하여 개인의 선호에 맞출 수 있는 유연성도 갖추고 있습니다.',
 'question_type': 'complex',
 'contexts': '프랑스 요리로 더 살펴보면, 코코뱅은 닭고기를 레드와인에 장시간 끓여 만드는\n전통 요리입니다. 와인의 풍미가 닭고기에 깊이 배어들어 독특한 맛을 자아내며,\n양파와 버섯, 베이컨을 함께 넣어 풍성한 맛을 더합니다. 라타투이는 가지, 토마\n토, 호박, 파프리카 등 지중해 채소들을 올리브유로 천천히 익혀 만드는 프로방\n스 지방의 대표적인 채식 요리입니다. 끼슈는 버터가 듬뿍 들어간 파이 반죽 위\n에 달걀과 크림을 섞은 커스터드를 붓고 베이컨, 치즈, 채소를 넣어 구워내

# 3. Amazon Bedrock KB 생성

이 테스트 데이터셋은 문서 검색과 응답 생성을 결합하는 RAG 파이프라인에서 실제 쿼리를 시뮬레이션하는 데 사용됩니다.

여기서는 RAG 구성의 일부로 Amazon Bedrock의 KnowledgeBase에 대한 기본 설정을 사용하고 있습니다.

_1. 아래 코드로 테스트하려면 Amazon Bedrock 콘솔에서 KnowledgeBase가 사전 구성되어 있어야 합니다_

_2. 평가하고자 하는 특정 RAG 파이프라인이 있다면 아래 셀을 적절히 수정하시기 바랍니다_

## 3.1 컨텍스트를 KB로 부터 가져오기

In [7]:
# RAG implementation sample 1 (Replace with RAG pipeline for evaluation)
from libs.bedrock_kb_util import context_retrieval_from_kb

amazon_kb_id = 'RAILTN1YDB'

question = test_dataset[0]['question']
# question = test_dataset[1]['question']
print("Question:\n", question)
search_result = context_retrieval_from_kb(question, 3, 'us-west-2', amazon_kb_id, 'SEMANTIC')
# print("search_result[0]:", search_result[0])

contexts = "\n--\n".join([result['content'] for result in search_result])
print("Context from Amazon KB:")
print(format_text(contexts))
# print_formatted_text(contexts)

Question:
 지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성을 제안한다면, 어떤 요리들을 선택하고 그 이유는 무엇인가요?
Context from Amazon KB:
인도 요리로는 버터치킨이 있는데, 요거트로 마리네이드한 닭고기를 탄두르에 구 워 토마토와 버터가 들어간 크리미한 커리 소스에 조려냅니다.
팔락 파니르는 시 금치 커리에 인도식 치즈인 파니르를 넣어 만드는 채식 요리이고, 달 마크니는 렌틸콩을 버터와 크림으로 맛있게 조리한 요리입니다.중동 요리에서는 샤워마가 유명한데, 양고기나 닭고기를 특제 향신료에 재워 수직 으로 구워 얇게 썰어 피타빵에 싸먹습니다.
팔라펠은 병아리콩으로 만든 완자를 튀겨서 피타빵에 채소와 함께 넣어 먹습니다.
후무스는 병아리콩을 갈아서 타히 니, 레몬즙, 올리브오일과 함께 섞어 만드는 디핑소스입니다.
한식으로 돌아와서, 갈비찜은 소갈비를 간장 베이스의 달달한 양념으로 푹 조려내 는 요리입니다.
닭갈비는 닭고기를 고추장 양념에 채소와 함께 볶아 치즈를 올려 먹는 요리이고, 족발은 돼지 앞다리를 향신료와 함께 오래 삶아 쫄깃하게 만듭니 다.
일식을 더 보면, 규동은 소고기를 달달한 간장 소스로 조려서 밥 위에 올리는 덮 밥입니다.
라멘은 진한 육수에 중국식 면을 넣고 차슈와 면발, 죽은 달걀을 올려 먹는 국수이고, 우동은 굵은 밀가루 면을 간장 베이스의 국물에 담아 먹습니다.
중식으로는 양장피가 있는데, 해파리와 각종 채소, 고기를 참기름 소스에 무치는 요리입니다.
어향육슬은 돼지고기를 채썰어 매콤새콤한 소스로 볶아내고, 팔보채 는 해산물과 채소를 굴소스로 볶아내는 고급 요리입니다.
동남아 요리로, 나시고랭은 인도네시아식 볶음밥으로 새우페이스트로 맛을 내는 것이 특징입니다.
락사는 코코넛 커리 수프에 면을 넣어 먹는 싱가포르의 대표 요리이고, 할라이는 태국식 바비큐 치킨으로 코코넛밀크에 재워 구워냅니다.
서양요리로 더 들어가면, 비프 스트로가노프는 소고기를 버섯과 함께 사워크림 소 스에

## 3.2 KB로 부터 Context를 LLM 을 통한 답변 생성
여기서는 검색된 컨텍스트를 기반으로 답변을 생성하고 있습니다.

In [8]:
import boto3
from botocore.config import Config

# model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
region = 'us-west-2'

retry_config = Config(
    region_name=region,
    retries={"max_attempts": 10, "mode": "standard"}
)
boto3_client = boto3.client("bedrock-runtime", config=retry_config)
answer = generate_answer(question, contexts, boto3_client, model_id)
print("Question: \n", question)
print("Answer: ")
print_formatted_text(answer)


Question: 
 지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성을 제안한다면, 어떤 요리들을 선택하고 그 이유는 무엇인가요?
Answer: 
지중해 연안 국가들의 요리 중에서 채식주의자와 육식주의자를 모두 만족시킬 수 있는 메뉴로 다음과 같은 요리들을 추천할 수 있습니다:

1. 라타투이 (프랑스):
- 채식주의자를 위한 메뉴로, 가지, 토마토, 호박, 파프리카 등 지중해 채소들을 올리브유로 천천히 익혀 만드는 채식 요리입니다.
- 풍부한 채소와 올리브유로 만들어 영양가 높고 맛있는 요리입니다.

2. 수블라키 (그리스):
- 육식주의자를 위한 메뉴로, 돼지고기를 레몬과 허브로 마리네이드해 꼬치에 꿰어 구워 피타빵에 싸 먹는 요리입니다.
- 채식주의자의 경우 피타빵과 허브 마리네이드 소스를 이용해 채소나 식물성 단백질로 대체할 수 있습니다.

3. 가스파초 (스페인):
- 토마토와 오이, 피망을 갈아서 만드는 차가운 스프로, 채식주의자와 육식주의자 모두에게 적합합니다.
- 여름철 상큼하고 건강한 요리로, 필요시 고기나 해산물을 곁들일 수 있습니다.

이 메뉴들은 풍부한 채소, 허브, 올리브오일을 사용하여 건강하고 맛있으며, 개인의 식단에 따라 쉽게 변형할 수 있는 다재다능한 요리들입니다.


# 4. Text Dataset 의 모든 샘플 질문에 대해 KB로 부터 Context 를 얻고 LLM 통해 답변 얻기

이 섹션에서는 컨텍스트 검색부터 답변 생성까지 전체 파이프라인을 샘플 질문들에 대해 실행합니다.

### 4.1 파라미터 설정

In [25]:
kb_region = 'us-west-2'
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
# model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
kb_id = amazon_kb_id
top_k = 3

### 4.2 KB 로 부터 Context 얻고, LLM 답변 생성

In [26]:
from time import sleep

def process_item(item):
    sleep(5)  # Prevent throttling
    question = item['question']
    search_result = context_retrieval_from_kb(question, top_k, kb_region, kb_id, 'SEMANTIC')

    contexts = [result['content'] for result in search_result]
    answer = generate_answer(question, "\n--\n".join(contexts), boto3_client, model_id)

    return {
        'question': item['question'],
        'ground_truth': item['ground_truth'],
        'original_contexts': item['contexts'],
        'retrieved_contexts': contexts,
        'answer': answer
    }

updated_dataset = test_dataset.map(process_item, remove_columns=test_dataset.column_names)  # 기존의 모든 컬럼 제거)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map: 100%|██████████| 5/5 [01:21<00:00, 16.33s/ examples]


In [27]:
updated_dataset

Dataset({
    features: ['question', 'ground_truth', 'original_contexts', 'retrieved_contexts', 'answer'],
    num_rows: 5
})

# 5. RAGAS Rag 평가 수행

## 5.1. RAGAS 래핑 모델 생성

In [28]:
# model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
# emb_model_id = "us.amazon.nova-micro-v1:0"
emb_model_id = "amazon.titan-embed-text-v2:0"

In [29]:
import boto3
from datasets import Dataset

from langchain_aws import ChatBedrockConverse
from ragas import evaluate

# Bedrock 클라이언트 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)

from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatBedrockConverse(
    model= model_id, 
    client=bedrock_client,
))

from langchain_community.embeddings import BedrockEmbeddings
# from ragas.embeddings import LangchainEmbeddingWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Bedrock Embeddings 설정
base_embeddings = BedrockEmbeddings(
    client=bedrock_client,
    model_id= emb_model_id
)

# RAGAS Wrapper로 감싸기
evaluator_embeddings = LangchainEmbeddingsWrapper(base_embeddings)

### 5.2. 평가용 데이터셋 생성

In [30]:
def prepare_ragas_evaluation_dataset(example):
    return {
        "question": example["question"],
        "answer": example["answer"],
        "contexts": example["retrieved_contexts"],
        "ground_truth": example["ground_truth"]
    }

data = [{
    "question": "파이썬이란 무엇인가요?",
    "ground_truth": "파이썬은 쉽고 간결한 프로그래밍 언어입니다.",
    "retrieved_contexts": ["파이썬은 프로그래밍 언어입니다.", "파이썬은 읽기 쉽고 간결합니다."],
    "generated_answer": "파이썬은 읽기 쉽고 간결한 프로그래밍 언어입니다."
}]
# dataset = Dataset.from_list(data)


# 2. RAGAS 평가용 데이터셋 포맷으로 변환
eval_dataset = updated_dataset.map(
    prepare_ragas_evaluation_dataset,
    remove_columns=updated_dataset.column_names
)
eval_dataset


Map: 100%|██████████| 5/5 [00:00<00:00, 1073.15 examples/s]


Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 5
})

### 5.2. 평가 메트릭으로 정의 후 실행

In [31]:
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextRecall,
    ContextPrecision
)
# 먼저 Faithfulness만 테스트
try:
    metrics = [
                Faithfulness(llm=evaluator_llm),
                AnswerRelevancy(llm=evaluator_llm, embeddings= evaluator_embeddings),
                ContextRecall(llm=evaluator_llm),
                ContextPrecision(llm=evaluator_llm),
    ]
    
    # 평가 실행a
    results = evaluate(
        eval_dataset,
        metrics=metrics,
        # show_individual_scores=True  # 이 부분이 추가됨
    )
    
except Exception as e:
    print(f"Error occurred: {str(e)}")
    print(f"Error type: {type(e)}")

results

Evaluating: 100%|██████████| 20/20 [02:02<00:00,  6.11s/it]


{'faithfulness': 0.6141, 'answer_relevancy': 0.3492, 'context_recall': 0.7584, 'context_precision': 0.7000}

In [34]:
result_df = results.to_pandas()
result_df

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision
0,지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴...,"[인도 요리로는 버터치킨이 있는데, 요거트로 마리네이드한 닭고기를 탄두르에 구 워 ...",주어진 맥락을 바탕으로 지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를...,지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴...,0.416667,0.443110,1.000000,0.5
1,팟타이는 어떤 재료로 만드나요?,"[인도 요리로는 버터치킨이 있는데, 요거트로 마리네이드한 닭고기를 탄두르에 구 워 ...",팟타이는 다음과 같은 재료로 만듭니다:\n\n1. 쌀국수\n2. 새우\n3. 땅콩가...,팟타이는 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려 만듭니다.,0.857143,0.170719,1.000000,0.5
2,"한국, 이탈리아, 태국, 베트남 요리의 특징을 비교하며, 각 나라의 대표 요리에서 ...","[볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께 볶다...","주어진 맥락을 바탕으로 한국, 이탈리아, 태국, 베트남 요리의 특징과 공통점을 비교...","한국, 이탈리아, 태국, 베트남 요리는 각각 고유한 특징을 가지고 있지만, 몇 가지...",0.666667,0.602608,0.363636,1.0
3,라자냐는 어떤 재료로 만들어지나요?,"[볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께 볶다...",라자냐는 다음과 같은 재료로 만들어집니다:\n\n1. 파스타 면\n2. 미트소스\n...,"라자냐는 파스타면, 미트소스, 베샤멜 소스, 치즈를 층층이 쌓아 오븐에 구워 만듭니다.",0.857143,0.225613,1.000000,1.0
4,"여러 나라의 요리법을 비교했을 때, 와인을 주요 재료로 사용하는 요리들의 공통점과 ...",[끼슈는 버터가 듬뿍 들어간 파이 반죽 위 에 달걀과 크림을 섞은 커스터드를 붓고 ...,주어진 맥락에서 와인을 주요 재료로 사용하는 요리들에 대한 정보를 찾아보면 다음과 ...,"와인을 주요 재료로 사용하는 요리들을 비교해보면, 프랑스의 비프 부르기뇽과 코코뱅이...",0.272727,0.303852,0.428571,0.5


### 5.3. 각각의 결과 보기

In [37]:

show_sample_row_eval(result_df, row=0)


## Question: 
 지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성을 제안한다면, 어떤 요리들을 선택하고 그 이유는 무엇인가요?
## Ground_truth: 
 지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성은 다음과 같습니다:

1. 프랑스의 라타투이: 가지, 토마토, 호박, 파프리카 등 다양한 채소로 만든 채식 요리로, 채식주의자에게 적합합니다.

2. 스페인의 가스파초: 토마토, 오이, 피망으로 만든 차가운 스프로, 채식주의자도 즐길 수 있는 상큼한 요리입니다.

3. 그리스의 무사카: 가지와 다진 고기를 층층이 쌓아 만드는 요리로, 육식주의자를 위한 선택입니다. 채식 버전으로 변형도 가능합니다.

4. 모로코의 쿠스쿠스: 밀가루로 만든 알갱이에 채소를 얹어 먹는 요리로, 고기를 별도로 제공하여 선택적으로 먹을 수 있게 합니다.

5. 스페인의 빠에야: 해산물과 채소를 넣은 쌀 요리로, 해산물을 먹는 사람과 채식주의자 모두에게 적합합니다.

이러한 구성은 다양한 식단 선호도를 고려하면서도 지중해 요리의 특색을 잘 보여줍니다. 채식 옵션과 육류 옵션을 동시에 제공하여 모든 식성의 사람들을 만족시킬 수 있으며, 각 요리의 재료와 조리법을 조절하여 개인의 선호에 맞출 수 있는 유연성도 갖추고 있습니다.
## Generated Answer: 
 주어진 맥락을 바탕으로 지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성을 제안하겠습니다:

1. 스페인의 가스파초: 토마토, 오이, 피망으로 만든 차가운 스프로, 채식주의자에게 적합합니다.

2. 그리스의 돌마데스: 포도잎에 밥과 다진 고기를 싸서 만드는 요리로, 고기 대신 채소로 채워 채식 버전으로 만들 수 있습니다.

3. 프랑스의 라타투이: 가지, 토마토, 호박, 파프리카 등 다양한 채소로 만든 프로방스 지방의 대표적인 채식 요리입니다.

4. 스페인의 빠에야: 해산물과 채소를 넣은 쌀

# 6. Custom RAG Evaluation 수행

이제 다양한 지표를 기반으로 시스템을 평가하겠습니다.

자세한 구현 내용은 `libs/custom_ragas.py` 파일을 참조하시기 바랍니다.

이 스크립트에는 정확도와 관련성 등 여러 측면에서 RAG 파이프라인의 성능을 평가하는데 사용되는 구체적인 평가 기준이 포함되어 있습니다.

### 6.1. 평가용 데이터셋 생성

In [38]:
from libs.custom_ragas import (
    evaluate,
    AnswerRelevancy, 
    Faithfulness, 
    ContextRecall,
    ContextPrecision
)

# llm_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
llm_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
# llm_id = "us.amazon.nova-micro-v1:0"
emb_id = "amazon.titan-embed-text-v2:0"
region = "us-west-2"

# metrics = [Faithfulness]
metrics = [AnswerRelevancy, Faithfulness, ContextRecall, ContextPrecision]

def map_dataset(example):
    return {
        "user_input": example["question"],
        "retrieved_contexts": example["retrieved_contexts"],
        "referenced_contexts": example["original_contexts"],
        "response": example["answer"],
        "reference": example["ground_truth"]
    }

dataset = updated_dataset.map(map_dataset, remove_columns=updated_dataset.column_names)  # 기존의 모든 컬럼 제거))


Map: 100%|██████████| 5/5 [00:00<00:00, 1001.98 examples/s]


In [39]:
dataset

Dataset({
    features: ['retrieved_contexts', 'user_input', 'referenced_contexts', 'response', 'reference'],
    num_rows: 5
})

### 6.2. 평가 메트릭으로 정의 후 실행

In [40]:
results = evaluate(dataset, metrics, llm_id, emb_id, region)
# results = evaluate(dataset, metrics, llm_id, emb_id, region, verbose=True)

print("Average Scores:")
print(results['average_scores'])

print("\nDetailed Results:")
for row in results['detailed_results']:
    print(row)

AnswerRelevancy - Row 1: Score = 0.3837319994407658
Faithfulness - Row 1: Score = 0.6666666666666666
ContextRecall - Row 1: Score = 0.8571428571428571
ContextPrecision - Row 1: Score = 0.99999999995
AnswerRelevancy - Row 2: Score = 0.9295958672903519
Faithfulness - Row 2: Score = 1.0
ContextRecall - Row 2: Score = 0.3333333333333333
ContextPrecision - Row 2: Score = 0.49999999995
AnswerRelevancy - Row 3: Score = 0.46601507931996994
Faithfulness - Row 3: Score = 0.8571428571428571
ContextRecall - Row 3: Score = 0.8
ContextPrecision - Row 3: Score = 0.9999999999
AnswerRelevancy - Row 4: Score = 0.9180536253265653
Faithfulness - Row 4: Score = 1.0
ContextRecall - Row 4: Score = 1.0
ContextPrecision - Row 4: Score = 0.9999999999
AnswerRelevancy - Row 5: Score = 0.47347188129307943
Faithfulness - Row 5: Score = 0.25
ContextRecall - Row 5: Score = 1.0
ContextPrecision - Row 5: Score = 0.49999999995
Average Scores:
{'AnswerRelevancy': 0.6341736905341465, 'Faithfulness': 0.7547619047619047, 'C

### 6.3. 결과를 JSON 에 저장

In [41]:
import json
import json

def save_json_results(results, json_filename="../data/sample_ragas_result.json"):
    json_results = {
        'average_scores': results['average_scores'],
        'detailed_results': results['detailed_results']
    }

    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(json_results, f, ensure_ascii=False, indent=4)

    print(f"Results saved to {json_filename}")
    print(json_results)
    
    return json_results

json_filename="../data/sample_ragas_result.json"
saved_results = save_json_results(results, json_filename)

saved_results = save_json_results(results)
# Or with a custom filename:
# saved_results = save_json_results(results, "custom_path/results.json")


Results saved to ../data/sample_ragas_result.json
{'average_scores': {'AnswerRelevancy': 0.6341736905341465, 'Faithfulness': 0.7547619047619047, 'ContextRecall': 0.7980952380952381, 'ContextPrecision': 0.79999999993}, 'detailed_results': [{'row': 1, 'AnswerRelevancy': 0.3837319994407658, 'Faithfulness': 0.6666666666666666, 'ContextRecall': 0.8571428571428571, 'ContextPrecision': 0.99999999995}, {'row': 2, 'AnswerRelevancy': 0.9295958672903519, 'Faithfulness': 1.0, 'ContextRecall': 0.3333333333333333, 'ContextPrecision': 0.49999999995}, {'row': 3, 'AnswerRelevancy': 0.46601507931996994, 'Faithfulness': 0.8571428571428571, 'ContextRecall': 0.8, 'ContextPrecision': 0.9999999999}, {'row': 4, 'AnswerRelevancy': 0.9180536253265653, 'Faithfulness': 1.0, 'ContextRecall': 1.0, 'ContextPrecision': 0.9999999999}, {'row': 5, 'AnswerRelevancy': 0.47347188129307943, 'Faithfulness': 0.25, 'ContextRecall': 1.0, 'ContextPrecision': 0.49999999995}]}
Results saved to ../data/sample_ragas_result.json
{'av

### 6.4 최종 결과 보이기

In [42]:
# 사용 예시
results_path = json_filename
merged_df = merge_dataset_and_results(updated_dataset, results_path)
print("Average Scores:")
print(results['average_scores'])

merged_df


Average Scores:
{'AnswerRelevancy': 0.6341736905341465, 'Faithfulness': 0.7547619047619047, 'ContextRecall': 0.7980952380952381, 'ContextPrecision': 0.79999999993}


,question,ground_truth,original_contexts,retrieved_contexts,answer,AnswerRelevancy,Faithfulness,ContextRecall,ContextPrecision
0,지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴...,지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴...,"프랑스 요리로 더 살펴보면, 코코뱅은 닭고기를 레드와인에 장시간 끓여 만드는\n전통...","[인도 요리로는 버터치킨이 있는데, 요거트로 마리네이드한 닭고기를 탄두르에 구 워 ...",주어진 맥락을 바탕으로 지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를...,0.383732,0.666667,0.857143,1.0
1,팟타이는 어떤 재료로 만드나요?,팟타이는 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려 만듭니다.,기와 달걀을 간장 베이스의 달달한 육수에 조려 밥 위에 올립니다.\n중식으로는 마파...,"[인도 요리로는 버터치킨이 있는데, 요거트로 마리네이드한 닭고기를 탄두르에 구 워 ...",팟타이는 다음과 같은 재료로 만듭니다:\n\n1. 쌀국수\n2. 새우\n3. 땅콩가...,0.929596,1.000000,0.333333,0.5
2,"한국, 이탈리아, 태국, 베트남 요리의 특징을 비교하며, 각 나라의 대표 요리에서 ...","한국, 이탈리아, 태국, 베트남 요리는 각각 고유한 특징을 가지고 있지만, 몇 가지...","볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다...","[볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께 볶다...","주어진 맥락을 바탕으로 한국, 이탈리아, 태국, 베트남 요리의 특징과 공통점을 비교...",0.466015,0.857143,0.800000,1.0
3,라자냐는 어떤 재료로 만들어지나요?,"라자냐는 파스타면, 미트소스, 베샤멜 소스, 치즈를 층층이 쌓아 오븐에 구워 만듭니다.",볶다가 뜨거운 육수를 조금씩 부어가며 천천히 익혀 만듭니다. 라자냐는 파스타\n면과...,"[볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께 볶다...",라자냐는 다음과 같은 재료로 만들어집니다:\n\n1. 파스타 면\n2. 미트소스\n...,0.918054,1.000000,1.000000,1.0
4,"여러 나라의 요리법을 비교했을 때, 와인을 주요 재료로 사용하는 요리들의 공통점과 ...","와인을 주요 재료로 사용하는 요리들을 비교해보면, 프랑스의 비프 부르기뇽과 코코뱅이...","요리입니다. 어향육슬은 돼지고기를 채썰어 매콤새콤한 소스로 볶아내고, 팔보채\n는 ...",[끼슈는 버터가 듬뿍 들어간 파이 반죽 위 에 달걀과 크림을 섞은 커스터드를 붓고 ...,주어진 맥락에서 와인을 주요 재료로 사용하는 요리들에 대한 정보를 찾아보면 다음과 ...,0.473472,0.250000,1.000000,0.5
